Treinando uma rede neural usando pytorch
========================================



## Introdução



O notebook anterior foi o *season finale* da construção da rede neural usando Python puro. Este notebook é o epílogo.

Aqui veremos como podemos construir e treinar uma rede neural usando `pytorch`, um módulo especializado em redes neurais artificiais.

Motivos para usarmos o `pytorch`:

1.  A `MLP` que criamos em Python puro não é otimizada. Tente treinar uma rede com uns 100 neurônios em uma camada e verá como ela é bem lenta! `pytorch` é escrito por diversos programadores que além de primar pelas contas estarem corretas, eles fazem de tudo para que elas ocorram de forma eficiente.

2.  `pytorch` já tem praticamente tudo que precisamos implementado! Quer calcular o gradiente local da função arco tangente? Ele já tem! Quer usar a função de ativação ReLU? Ele já tem!

3.  `pytorch` tem suporte a treinar uma rede neural usando processamento gráfico (GPU). Isso acelera muito o treino de redes neurais artificiais complexas. Temos GPUs para usarmos no HPC da Ilum!

4.  `pytorch` é usado tanto na academia quanto no mundo corporativo. Junto com `tensorflow` são os dois módulos estado da arte para o projeto e treino de redes neurais.



## Objetivo



Treinar uma rede neural artificial tipo Multilayer Perceptron usando `pytorch`.



## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Código e discussão



### Dados



Vamos treinar nossa rede neural com nosso velho amigo, o dataset de diamantes! Como todo processo de treino de aprendizado de máquina, precisamos reservar um conjunto de dados para treino e outro para teste.



In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

Redes neurais costumam se dar bem quando os dados estão entre $[0, 1]$ ou entre $[-1, 1]$. Redes neurais costumam não se dar bem quando os dados estão em escalas muito diferentes. Por conta disso, normalizar os dados antes de prosseguir é recomendado! Vamos usar o `MinMaxScaler` do `scikit-learn` que aprendemos semestre passado.



In [3]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

### Tensores



Na nossa rede neural feita em Python puro, a base de tudo era a classe `Valor`. Para o `pytorch`, a base de tudo são os tensores! Tensores são objetos que armazenam dados de forma similar aos arrays de `numpy`, porém registram todas as operações realizadas para o cálculo do gradiente local (assim como nós fizemos com a classe `Valor`).



In [4]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

Vamos checar os dados para ver como estão.



In [5]:
print(X_treino)
print()
print(y_treino)

tensor([[0.0686, 0.5611, 0.2885, 0.4777, 0.0876, 0.1022],
        [0.0624, 0.5250, 0.2500, 0.4730, 0.0871, 0.0994],
        [0.0478, 0.4917, 0.2692, 0.4572, 0.0829, 0.0934],
        ...,
        [0.1185, 0.5333, 0.2692, 0.5410, 0.0995, 0.1142],
        [0.0166, 0.5139, 0.2885, 0.3929, 0.0708, 0.0811],
        [0.0333, 0.5361, 0.2692, 0.4209, 0.0776, 0.0890]])

tensor([[0.0676],
        [0.0586],
        [0.0434],
        ...,
        [0.1527],
        [0.0172],
        [0.0216]])


### Criando a rede neural



Para criar uma rede neural usando `pytorch` é necessário criar uma classe. Observe aqui que a classe criada é baseada na classe `nn.Module` do `pytorch`. Classes podem ser baseadas em outras classes! Trata-se da característica de herança das classes.

Observe que definimos as camadas da rede neural dentro de um objeto `nn.Sequential`.

Observe também o método `forward`. Este funciona de forma similar ao dunder `__call__` que vimos anteriormente. O `pytorch` requer que usemos o `forward` e não o `__call__`, então aceitamos e seguimos em frente.



In [6]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

Agora podemos criar a nossa rede neural!



In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

E podemos checar os parâmetros internos dela!



In [8]:
for p in minha_MLP.parameters():
    print(p)

Parameter containing:
tensor([[-0.0218,  0.2822, -0.2330,  0.0462,  0.0093,  0.1916],
        [-0.1960, -0.2350,  0.2680,  0.3098,  0.2810,  0.2272],
        [-0.1146, -0.3035, -0.2286,  0.3033, -0.2723,  0.1009],
        [ 0.1298,  0.1967,  0.1586, -0.0394, -0.1731,  0.2722],
        [ 0.3964, -0.0886, -0.4026,  0.0320, -0.2212,  0.2930],
        [-0.0263,  0.2137, -0.3749, -0.1336,  0.3834,  0.0974],
        [-0.0693, -0.3991,  0.2072,  0.3612, -0.3226, -0.2516],
        [ 0.3214, -0.1591, -0.3250, -0.3903,  0.2844, -0.0796],
        [ 0.0911, -0.2325,  0.2815,  0.2685,  0.3747, -0.1280],
        [-0.1228, -0.2010,  0.1920,  0.1910,  0.1631, -0.0850],
        [ 0.2102,  0.1308,  0.0773, -0.2090,  0.1284,  0.1094],
        [-0.0114, -0.0841, -0.2459,  0.0370, -0.2182,  0.3902],
        [ 0.1830,  0.1742,  0.2230,  0.1922, -0.2269,  0.3329],
        [-0.0434,  0.0944,  0.3866, -0.3959,  0.2582, -0.3964],
        [-0.1703, -0.0877,  0.0443,  0.1376, -0.1793, -0.3310],
        [-0.1857, 

Também podemos realizar uma previsão, mas ela provavelmente será bem ruim!



In [9]:
y_prev = minha_MLP(X_treino)
y_prev

tensor([[0.1147],
        [0.1152],
        [0.1165],
        ...,
        [0.1116],
        [0.1203],
        [0.1183]], grad_fn=<AddmmBackward0>)

### A função de perda e o otimizador



O `pytorch` já vem com diversas funções de perda já implementadas. A que computa o erro quadrático médio, por exemplo, já está pronta!

Fora isso, precisamos definir o nosso otimizador. O otimizador é quem cuida de atualizar os parâmetros da nossa rede. Nós implementamos na nossa rede em Python puro a descida do gradiente. Aqui usaremos um otimizador que é uma modificação da descida do gradiente, chamado `Adam`. Trata-se, simplificadamente, de uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro e que se altera ao longo do aprendizado. `Adam` é um poderosíssimo otimizador! Não tem porque não usarmos ele neste caso.



In [10]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

### O treino da rede



Para avisar o `pytorch` que iremos treinar a rede, vamos invocar o método `train` da nossa rede. Toda rede recém-criada já está no modo treino, mas aqui deixarei explícito para ficar bem claro.



In [11]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Observe que o ciclo do treino tem as mesmas etapas que vimos no notebook anterior, nada de novo! Aqui basta observar como executamos essas etapas no `pytorch`.



In [13]:
NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.0055)
1 tensor(0.0055)
2 tensor(0.0055)
3 tensor(0.0055)
4 tensor(0.0055)
5 tensor(0.0055)
6 tensor(0.0055)
7 tensor(0.0055)
8 tensor(0.0055)
9 tensor(0.0055)
10 tensor(0.0055)
11 tensor(0.0055)
12 tensor(0.0055)
13 tensor(0.0055)
14 tensor(0.0055)
15 tensor(0.0055)
16 tensor(0.0055)
17 tensor(0.0055)
18 tensor(0.0055)
19 tensor(0.0055)
20 tensor(0.0055)
21 tensor(0.0055)
22 tensor(0.0055)
23 tensor(0.0055)
24 tensor(0.0055)
25 tensor(0.0055)
26 tensor(0.0055)
27 tensor(0.0055)
28 tensor(0.0055)
29 tensor(0.0055)
30 tensor(0.0055)
31 tensor(0.0055)
32 tensor(0.0055)
33 tensor(0.0055)
34 tensor(0.0055)
35 tensor(0.0055)
36 tensor(0.0055)
37 tensor(0.0055)
38 tensor(0.0055)
39 tensor(0.0055)
40 tensor(0.0055)
41 tensor(0.0055)
42 tensor(0.0055)
43 tensor(0.0055)
44 tensor(0.0055)
45 tensor(0.0055)
46 tensor(0.0055)
47 tensor(0.0055)
48 tensor(0.0055)
49 tensor(0.0055)
50 tensor(0.0055)
51 tensor(0.0055)
52 tensor(0.0055)
53 tensor(0.0055)
54 tensor(0.0055)
55 tensor(0.0055)
56

Após o treino, podemos checar a performance da nossa rede. Observe a linha que inicia com `with`. Tudo que estiver no bloco do `with torch.no_grad()` é computado normalmente, porém o grafo computacional e os gradientes locais não são computados. Use isso sempre que for realizar contas com tensores fora do treino!!



In [14]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[1577.99996901] [1581.95348358]
[1409.99999118] [1538.41645336]
[1128.99997044] [1110.23162842]
[12991.99967575] [12017.03258133]
[14720.0001564] [14892.71289062]
[2938.99994564] [2766.69346333]
[6635.00014687] [7686.68050003]
[11215.99973297] [9701.02187729]
[1040.99999189] [1540.59957504]
[5040.00003624] [5184.31372261]
[9541.99988747] [8488.51317406]
[3792.99988461] [5518.7527504]
[1002.0000155] [756.09213924]
[2542.00005341] [1599.84289932]
[6788.00001717] [6577.45238686]
[9386.00025749] [7282.28456306]
[1245.99996853] [1725.74130249]
[997.00000262] [728.22033691]
[708.00000262] [760.7669301]
[1435.99997544] [1504.48174095]
[2590.00006676] [2595.66389084]
[1392.99997497] [1773.44999409]
[11755.99946976] [10153.26017761]
[2007.99997377] [1410.61798191]
[2707.99998474] [2644.57121849]
[631.00000417] [660.09969187]
[652.99999881] [1031.02132607]
[2492.00006247] [2633.57209969]
[594.99999416] [538.8190279]
[760.0000056] [1140.36565304]
[1179.00003028] [1112.34102392]
[1789.9999361] [18

### O teste da rede



Vamos supor que você já testou diversas arquiteturas de rede e está pronto para usar os dados de teste. Nesta etapa precisamos indicar para o `pytorch` que o treino acabou e estamos na etapa de avaliação. Fazemos isso rodando o método `eval`.



In [15]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Agora podemos realizar o teste!



In [16]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[523.00000858] [830.2299428]
[628.00000334] [638.6036191]
[442.00000328] [731.00401688]
[2284.99997044] [2537.48222351]
[1040.99999189] [861.92875385]
[809.99999654] [1212.09240389]
[3049.99987316] [3471.39913368]
[720.00000596] [713.86895084]
[877.99999249] [616.48411226]
[1292.00000429] [1720.71055889]
[7056.00004578] [4895.95920563]
[4176.99999142] [5728.95310974]
[13701.00020599] [14512.55812073]
[2948.0000515] [3957.85795593]
[16732.99985504] [14483.49539948]
[1815.00000048] [1938.92350674]
[16220.00016022] [12227.79077911]
[5967.00019073] [4613.38675022]
[473.00000042] [636.91081095]
[1078.9999795] [1034.95968199]
[452.00000319] [684.87968016]
[801.99998283] [781.77751255]
[2458.99994993] [2512.68928146]
[2492.99998236] [2803.20983219]
[582.00000203] [699.7570715]
[1405.9999671] [1470.36898327]
[8715.99991035] [11526.28358459]
[7634.00004578] [6594.47968864]
[506.99999839] [685.93499804]
[665.99999094] [753.85610008]
[1815.00000048] [1608.54727364]
[707.00001383] [715.37131119]
[

E, finalmente, computar alguma métrica para medir a performance do nosso modelo.



In [17]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 1376.708350204584


## Conclusão

Após o _season finale_, nos resta apenas um episódio extra: neste ntoebook, aprendemos a utilizar a biblioteca `pytorch`, um módulo especializado em redes neurais artificiais! Relembramos todo o processo utilizado no semestre anterior para <span style=color:red; text-size:150%>Machine Learning</span>; tratar os dados, normalizá-los, treinar o método de aprendizado, rodá-lo, e utilizar algum método para medir a sua precisão.

Para as RNAs, no entando, precisamos ainda utilizar o equivalente à nosa classe Ǜalores`dos notebooks anterires; os tensores! Para isso, 

## Playground

